# Adapted from https://dfm.io/posts/stan-c++/

Code updated to use CmdStanPy and recent `stan::math` conventions

In [ ]:
# First, import all the packages that we'll need:

import time
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import corner

import cmdstanpy

# Download the dataset from the Exoplanet Archive:
url = "https://exoplanetarchive.ipac.caltech.edu/data/ExoData/0113/0113357/data/UID_0113357_RVC_001.tbl"
r = requests.get(url)
if r.status_code != requests.codes.ok:
    r.raise_for_status()
data = np.array([l.split() for l in r.text.splitlines()
                 if not l.startswith("\\") and not l.startswith("|")],
                dtype=float)
t, rv, rv_err = data.T
t -= np.mean(t)

# Plot the observations "folded" on the published period:
# Butler et al. (2006) https://arxiv.org/abs/astro-ph/0607493
lit_period = 4.230785
plt.errorbar((t % lit_period)/lit_period, rv, yerr=rv_err, fmt=".k", capsize=0)
plt.xlim(0, 1)
plt.ylim(-110, 110)
plt.annotate("period = {0:.6f} days".format(lit_period),
             xy=(1, 0), xycoords="axes fraction",
             xytext=(-5, 5), textcoords="offset points",
             ha="right", va="bottom", fontsize=12)
plt.ylabel("radial velocity [m/s]")
plt.xlabel("phase");


In [ ]:
user_header = '''
#include <stan/math.hpp>
#include <ostream>
#include <sstream>
#include <stdexcept>

using stan::math::var;
using std::sin;
using std::cos;

inline double kepler (double M, double e, std::ostream* pstream) {
  // Check for un-physical parameters.
  if (e < 0.0 || e >= 1.0)
  {
    std::stringstream errmsg_stream__;
    stan::math::stan_print(&errmsg_stream__,
        "kepler eccentricity is ");
    stan::math::stan_print(&errmsg_stream__, e);
    throw std::domain_error(errmsg_stream__.str());
  }
  
  double E0 = M, E = M;
  for (int i = 0; i < 200; ++i) {
    double g = E0 - e * sin(E0) - M,
           gp = 1.0 - e * cos(E0);
    E = E0 - g / gp;

    // Check for convergence.
    if (abs((E - E0) / E) <= 1.234e-10) return E;

    E0 = E;
  }
  
  // If we get here, we didn't converge, but return the best estimate.
  return E;
}


inline var kepler (const var& M, const var& e, std::ostream* pstream) {
  // First, compute the value of E at the current values of M and e

  var E = kepler(M.val(), e.val(), pstream);
  
  // The set up the reverse-pass callback
  stan::math::reverse_pass_callback([E, M, e]() mutable {
    double dE_dM = 1.0 / (1.0 - e.val() * cos(E.val()));
    M.adj() += E.adj() * dE_dM;
    double dE_de = sin(E.val()) * dE_dM;
    e.adj() += E.adj() * dE_de;
  });

  return E;
}

// var, double and double, var overloads omitted

inline var kepler (double M, const var& e, std::ostream* pstream) {
  var E = kepler(M, e.val(), pstream);
  
  stan::math::reverse_pass_callback([E, e]() mutable {
    double E_val = E.val();
    double dE_de = sin(E_val) / (1.0 - e.val() * cos(E_val));
    e.adj() += E.adj() * dE_de;
  });
  
  return E;
}

inline var kepler (const var& M, double e, std::ostream* pstream) {
  var E = kepler(M.val(), e, pstream);
  
  stan::math::reverse_pass_callback([E, M, e]() mutable {
    M.adj() += E.adj() / (1.0 - e * cos(E.val()));
  });
  
  return E;
}

'''

In [ ]:
with open('exo-header.hpp', 'w') as f:
    f.writelines(user_header)

In [ ]:
model_code = """
functions {
    real kepler(real M, real e);
}
data {
    int<lower=0> N;          // number of times
    vector[N]    t;          // the times
    vector[N]    rv;         // the rvs
    vector[N]    rv_err;     // the rv uncertainties
    int<lower=0> N_pred;     // number of predictions
}
parameters {
    real<lower=0, upper=log(200)> log_K;       // ln(semi-amplitude)
    real<lower=0, upper=log(100)> log_P;       // ln(period)
    real<lower=0, upper=2*pi()>   phi;         // phase
    real<lower=0, upper=1>        e;           // eccentricity
    unit_vector[2]                w_vec;       // [cos(w), sin(w)]
    real<lower=-10, upper=5>      log_jitter;  // ln(jitter)
    real<lower=-10, upper=10>     rv0;         // mean RV
    real<lower=-10, upper=10>     rv_trend;    // RV trend
}
transformed parameters {
    real n       = 2*pi()*exp(-log_P);
    real K       = exp(log_K);
    real w       = atan2(w_vec[2], w_vec[1]);
    real jitter2 = exp(2*log_jitter);
    real t0      = (phi + w) / n;
    
    // This will store the predicted RV signal at the observed times
    vector[N] mod;
    
    // And this stores the RV trend at the observed times
    vector[N] trend;
    
    for (j in 1:N) {
        // Compute the predicted RV signal at time t[j]
        real M = n * t[j] - (phi + w);
        real E = kepler(M, e);
        real f = 2*atan2(sqrt(1+e) * tan(0.5*E), sqrt(1-e));
        trend[j] = rv0 + rv_trend * t[j] / 365.25;
        mod[j] = trend[j] + K * (w_vec[1]*(cos(f)+e)-w_vec[2]*sin(f));
    }
}
model {
    // We already have a uniform prior on "e" but it must be in the
    // range [0, 1].
    e ~ uniform(0, 1);
    
    // Loop over observations
    for (j in 1:N) {
        // Compute the predicted RV signal at time t[j]
        // Assume Gaussian uncertainties with jitter
        rv[j] ~ normal(mod[j], sqrt(rv_err[j]^2 + jitter2));
    }
}
generated quantities {
    // At each accepted step in the MCMC, compute the predicted RV
    // curve for `N_pred` equally-spaced phases
    vector[N_pred] rv_pred;
    for (j in 1:N_pred) {
        real M = (2.0*pi()*(j-1))/(N_pred-1) - (phi + w);
        real E = kepler(M, e);
        real f = 2*atan2(sqrt(1+e) * tan(0.5*E), sqrt(1-e));
        rv_pred[j] = K * (w_vec[1]*(cos(f)+e)-w_vec[2]*sin(f));
    }
}
"""

In [ ]:
with open("exoplanet.stan", 'w') as f:
    f.writelines(model_code)

In [ ]:
%%time
model = cmdstanpy.CmdStanModel(stan_file='exoplanet.stan', user_header='exo-header.hpp')

In [ ]:
np.random.seed(42)

# Custom initialization function. Some parameters are randomized
def init_func():
    omega = np.random.uniform(0, 2*np.pi)
    return dict(
        log_K=np.log(0.5 * (np.max(rv) - np.min(rv))),
        log_P=np.log(4.230785),
        phi=np.random.uniform(0, 2*np.pi),
        e=np.random.uniform(0, 0.01),
        w_vec=[np.cos(omega), np.sin(omega)],
        log_jitter=np.log(np.mean(rv_err)),
        rv0=np.mean(rv),
        rv_trend=0.0,
    )

# Run 5 randomized restarts to find the MAP parameters
data = dict(N=len(t), t=t, rv=rv, rv_err=rv_err, N_pred=100)
best = None
best_lp = -np.inf
for k in range(5):
    try:
        opt = model.optimize(data=data, inits=init_func(), sig_figs=10)
    except RuntimeError:
        continue
    if opt.optimized_params_dict['lp__'] > best_lp:
        best = opt
        best_lp = best.optimized_params_dict['lp__']
        
print(f"Maximum log-probability: {best_lp:.3f}")

In [ ]:
# Extract the best fit parameters
period = np.exp(best.log_P)
t0 = best.t0
pred = best.rv_pred
mod = best.mod
trend = best.trend

# Compute the effective uncertainties including the contribution
# of jitter
yerr = np.sqrt(rv_err**2 + np.exp(2*best.log_jitter))

# Plot the data, model, and residuals
plt.errorbar((t/period) % 1.0, rv - trend, yerr=yerr, fmt=".k", label="data", lw=1, ms=4)
plt.plot(np.arange(len(pred)) / (len(pred)-1), pred, label="MAP model")
plt.plot((t/period) % 1.0, rv - mod, ".k", alpha=0.3, mec="none",
         label="resiudals", ms=5)

plt.legend(loc=3, fontsize=11)
plt.xlim(0, 1)
plt.ylim(-110, 110)
plt.annotate("period = {0:.6f} days".format(period),
             xy=(1, 0), xycoords="axes fraction",
             xytext=(-5, 5), textcoords="offset points",
             ha="right", va="bottom", fontsize=12)
plt.ylabel("radial velocity [m/s]")
plt.xlabel("phase");

In [ ]:
%%time
# NB: initializing all chains to same place may compromise diagnostics
fit = model.sample(data=data, inits=best.stan_variables())

In [ ]:
fit.summary().head()

In [ ]:
print(fit.diagnose())

In [ ]:
params = ["log_P", "K", "e"]
samples = fit.draws_pd(vars=params)
samples["period"] = np.exp(samples["log_P"])
samples = np.array(samples[["period", "K", "e"]])
labels = ["period [days]", "semi-amplitude [m/s]", "eccentricity"]
corner.corner(samples, labels=labels)

In [ ]:
pred = fit.rv_pred
phase = np.arange(pred.shape[1]) / (pred.shape[1]-1)

q = np.percentile(pred, [16, 50, 84], axis=0)
plt.plot(phase, q[1], color="g")
plt.fill_between(phase, q[0], q[2], color="g", alpha=0.3)
period = np.exp(best.log_P)
plt.errorbar((t/period) % 1.0, rv - trend, yerr=yerr, fmt=".k", label="data", lw=1, ms=4)
plt.xlim(0, 1)
plt.ylim(-110, 110)
plt.annotate("period = {0:.6f} days".format(period),
             xy=(1, 0), xycoords="axes fraction",
             xytext=(-5, 5), textcoords="offset points",
             ha="right", va="bottom", fontsize=12)
plt.ylabel("radial velocity [m/s]")
plt.xlabel("phase");

In [ ]:
periods = np.exp(fit.log_P)
print("P = {0:.6f} ± {1:.6f}".format(np.mean(periods),  np.std(periods)))
amps = np.exp(fit.log_K)
print("K = {0:.2f} ± {1:.2f}".format(np.mean(amps),  np.std(amps)))